In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   validation_split=0.2)

In [4]:
train_generator = train_datagen.flow_from_directory(
        '../data/archive/train/eyes',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=32,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        '../data/archive/train/eyes',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=32,
        subset='validation')

Found 1162 images belonging to 2 classes.
Found 290 images belonging to 2 classes.


In [5]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(80, 80, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.5),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 78, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 39, 39, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [8]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=50,
      verbose=1,
      callbacks=early_stop)

Epoch 1/50
37/37 [==============================] - 24s 519ms/step - loss: 0.4530 - accuracy: 0.8141 - val_loss: 0.2715 - val_accuracy: 0.8759
Epoch 2/50
37/37 [==============================] - 4s 117ms/step - loss: 0.1301 - accuracy: 0.9561 - val_loss: 0.2010 - val_accuracy: 0.8897
Epoch 3/50
37/37 [==============================] - 4s 102ms/step - loss: 0.0810 - accuracy: 0.9690 - val_loss: 0.1282 - val_accuracy: 0.9448
Epoch 4/50
37/37 [==============================] - 5s 122ms/step - loss: 0.0598 - accuracy: 0.9742 - val_loss: 0.1662 - val_accuracy: 0.9310
Epoch 5/50
37/37 [==============================] - 5s 125ms/step - loss: 0.0468 - accuracy: 0.9828 - val_loss: 0.1670 - val_accuracy: 0.9414
Epoch 6/50
37/37 [==============================] - 5s 130ms/step - loss: 0.0411 - accuracy: 0.9836 - val_loss: 0.1990 - val_accuracy: 0.9414
Epoch 7/50
37/37 [==============================] - 4s 119ms/step - loss: 0.0530 - accuracy: 0.9811 - val_loss: 0.1469 - val_accuracy: 0.9448
Epoch

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
test_generator = test_datagen.flow_from_directory(
        '../data/test/yourfolderforcroppedeyes',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 45 images belonging to 2 classes.


In [12]:
model.evaluate(test_generator)

3/3 [==============================] - 1s 86ms/step - loss: 0.2465 - accuracy: 0.9111


[0.24646618962287903, 0.9111111164093018]

In [13]:
model.save("base_eyes_model.h5")